In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Prepare Data

In [2]:
tr = pd.read_csv("../data/train_with_feature.csv")
ts = pd.read_csv("../data/test_with_feature.csv")

In [3]:
tr.head(3)

,review,label,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,...,feature290,feature291,feature292,feature293,feature294,feature295,feature296,feature297,feature298,feature299
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3,0.228316,-0.020764,-0.001777,0.100460,-0.017000,-0.035074,0.036856,0.015120,...,-0.001877,0.045393,0.010673,-0.024315,0.011204,0.028232,-0.036892,-0.003593,-0.005051,-0.000360
1,อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา...,4,0.282241,-0.030783,-0.015046,0.039718,0.056262,-0.048687,-0.045608,0.015437,...,-0.016354,-0.020544,0.004761,0.009503,0.004973,-0.017164,-0.025104,-0.023011,0.012646,0.008128
2,ปอเปี๊ยะสด ทุกวันนี้รู้สึกว่าหากินยาก (ร้านที่...,3,0.286438,-0.032563,0.077864,0.004659,-0.024402,-0.038413,0.025931,0.006893,...,-0.001739,-0.000830,0.019939,-0.001312,-0.004875,-0.032898,0.033292,-0.002587,0.010849,0.020666


In [4]:
ts.head(3)

,reviewID,review,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,...,feature290,feature291,feature292,feature293,feature294,feature295,feature296,feature297,feature298,feature299
0,1,ร้านนี้จะอยู่เส้นสันกำแพง-แม่ออน เลยแยกบ่...,0.360498,0.581334,0.014321,-0.005751,-0.010322,-0.031789,-0.019707,0.036583,...,-0.003845,-0.018055,-0.019899,-0.015334,-0.017138,0.001517,-0.011955,0.015323,-0.004450,-0.005399
1,2,สั่งไป2 เมนู คือมัชฉะลาเต้ร้อน กับ ไอศครีมชาเข...,0.151875,-0.014623,-0.009261,0.030889,-0.034241,-0.017381,0.031627,0.025571,...,-0.008337,0.010086,-0.019927,0.028766,-0.002063,0.022431,0.026762,0.004502,0.004664,0.007936
2,3,ครัววงเดือน \n\nหิวดึกๆ ตระเวนหาร้านทาน มาเจอ...,0.313768,-0.025867,0.121377,0.146950,0.185460,0.069842,0.180548,-0.000253,...,0.009490,-0.039535,0.015618,-0.003521,0.002281,0.017251,0.034164,0.022599,0.032634,-0.019891


In [5]:
split = int(tr.shape[0]*9/10)
features = ["feature"+str(i) for i in range(300)]
x_train = tr[features][:split].as_matrix()
y_train = tr["label"][:split].as_matrix()
y_train_one_hot = pd.get_dummies(tr["label"][:split]).as_matrix()
x_val = tr[features][split:].as_matrix()
y_val = tr["label"][split:].as_matrix()
y_val_one_hot = pd.get_dummies(tr["label"][split:]).as_matrix()
x_test = ts[features].as_matrix()

print('Training data shape                  :', x_train.shape)
print('Training data labels shape           :', y_train.shape)
print('Training data labels one hot shape   :', y_train_one_hot.shape)
print('Validation data shape                :', x_val.shape)
print('Validation data labels shape         :', y_val.shape)
print('Validation data labels one hot shape :', y_val_one_hot.shape)
print('Test data shape                      :', x_test.shape)

Training data shape                  : (36000, 300)
Training data labels shape           : (36000,)
Training data labels one hot shape   : (36000, 5)
Validation data shape                : (4000, 300)
Validation data labels shape         : (4000,)
Validation data labels one hot shape : (4000, 5)
Test data shape                      : (6203, 300)


# Feedforward Nueral Network - Regression

In [116]:
from keras.models import Model
from keras.layers import Input, Dense, Dropout
from keras.optimizers import Adam

def get_nn_regression():
    input1 = Input(shape=(300,))
    x = Dense(100, activation='relu')(input1)
    x = Dropout(0.5)(x)
    x = Dense(10, activation='relu')(x)
    x = Dropout(0.1)(x)
    out = Dense(1, activation='relu')(x)
    model = Model(inputs=input1, outputs=out)
    model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['acc'])
    return model

In [117]:
from keras import backend as K
K.clear_session()

print('start training')
verbose = 1
model_nn_regression = get_nn_regression()

train_params = [(20, 32)]
for (epochs, batch_size) in train_params:
    print("train with {} epochs and {} batch size".format(epochs, batch_size))
    model_nn_regression.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose,
                           validation_data=(x_val, y_val))

start training
train with 20 epochs and 32 batch size
Train on 36000 samples, validate on 4000 samples
Epoch 1/20
36000/36000 [==============================] - 4s - loss: 2.0316 - acc: 0.3373 - val_loss: 0.5439 - val_acc: 0.5115
Epoch 2/20
36000/36000 [==============================] - 4s - loss: 0.9625 - acc: 0.3958 - val_loss: 0.5103 - val_acc: 0.5260
Epoch 3/20
36000/36000 [==============================] - 4s - loss: 0.8603 - acc: 0.4202 - val_loss: 0.5270 - val_acc: 0.5175
Epoch 4/20
36000/36000 [==============================] - 4s - loss: 0.7546 - acc: 0.4445 - val_loss: 0.5032 - val_acc: 0.5298
Epoch 5/20
36000/36000 [==============================] - 4s - loss: 0.6529 - acc: 0.4716 - val_loss: 0.5042 - val_acc: 0.5240
Epoch 6/20
36000/36000 [==============================] - 4s - loss: 0.5988 - acc: 0.4892 - val_loss: 0.4876 - val_acc: 0.5340
Epoch 7/20
36000/36000 [==============================] - 4s - loss: 0.5633 - acc: 0.5000 - val_loss: 0.4979 - val_acc: 0.5288
Epoch 8/

#### Calculate F1 score on Validation Data

In [118]:
from sklearn.metrics import f1_score
y_pred_val = model_nn_regression.predict(x_val)
y_pred_val = np.rint(y_pred_val).astype(int)
f1_score(y_val, y_pred_val,average='micro')

0.54674999999999996

#### Predict Rating For Test Data

In [119]:
y_pred = model_nn_regression.predict(x_test)
y_pred = np.rint(y_pred).astype(int)
y_pred.shape

(6203, 1)

In [120]:
pred = ts[["reviewID"]].copy()
pred["rating"] = y_pred

In [121]:
pred.head(10)

,reviewID,rating
0,1,4
1,2,4
2,3,3
3,4,4
4,5,4
5,6,4
6,7,4
7,8,3
8,9,4
9,10,4


In [122]:
pred.to_csv("../data/prediction_nn_regression.csv", index=False)

# Feedforward Nueral Network - Classification

In [6]:
from keras.models import Model
from keras.layers import Input, Dense, Dropout
from keras.optimizers import Adam

def get_nn_classification():
    input1 = Input(shape=(300,))
    x = Dense(100, activation='relu')(input1)
    x = Dropout(0.5)(x)
    x = Dense(10, activation='relu')(x)
    x = Dropout(0.1)(x)
    out = Dense(5, activation='softmax')(x)
    model = Model(inputs=input1, outputs=out)
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['acc'])
    return model

Using TensorFlow backend.


In [7]:
from keras import backend as K
K.clear_session()

print('start training')
verbose = 1
model_nn_classification = get_nn_classification()

train_params = [(20, 32)]
for (epochs, batch_size) in train_params:
    print("train with {} epochs and {} batch size".format(epochs, batch_size))
    model_nn_classification.fit(x_train, y_train_one_hot, epochs=epochs, batch_size=batch_size, verbose=verbose,
                           validation_data=(x_val, y_val_one_hot))

start training
train with 20 epochs and 32 batch size
Train on 36000 samples, validate on 4000 samples
Epoch 1/20
36000/36000 [==============================] - 16s - loss: 1.1706 - acc: 0.4891 - val_loss: 1.0278 - val_acc: 0.5357
Epoch 2/20
36000/36000 [==============================] - 5s - loss: 1.0512 - acc: 0.5265 - val_loss: 1.0026 - val_acc: 0.5455
Epoch 3/20
36000/36000 [==============================] - 5s - loss: 1.0295 - acc: 0.5331 - val_loss: 0.9989 - val_acc: 0.5490
Epoch 4/20
36000/36000 [==============================] - 5s - loss: 1.0211 - acc: 0.5365 - val_loss: 1.0004 - val_acc: 0.5397
Epoch 5/20
36000/36000 [==============================] - 5s - loss: 1.0129 - acc: 0.5391 - val_loss: 0.9962 - val_acc: 0.5505
Epoch 6/20
36000/36000 [==============================] - 5s - loss: 1.0027 - acc: 0.5421 - val_loss: 0.9942 - val_acc: 0.5513
Epoch 7/20
36000/36000 [==============================] - 5s - loss: 0.9943 - acc: 0.5491 - val_loss: 0.9951 - val_acc: 0.5490
Epoch 8

#### Calculate F1 score on Validation Data

In [8]:
from sklearn.metrics import f1_score
y_pred_distribution_val = model_nn_classification.predict(x_val)
y_pred_val = y_pred_distribution_val.argmax(axis=1)+1
f1_score(y_pred_val, y_val,average='micro')

0.54400000000000004

#### Predict Rating For Test Data

In [9]:
y_pred_distribution = model_nn_classification.predict(x_test)
y_pred = y_pred_distribution.argmax(axis=1)+1
y_pred.shape

(6203,)

In [10]:
pred = ts[["reviewID"]].copy()
pred["rating"] = y_pred

In [11]:
pred.head(10)

,reviewID,rating
0,1,4
1,2,4
2,3,3
3,4,4
4,5,3
5,6,4
6,7,4
7,8,3
8,9,4
9,10,4


In [12]:
pred.to_csv("../data/prediction_nn_classification.csv", index=False)